## Data

The following datasets are utilized in this analysis for calculating and mapping crop productivity over the past years:

1. **Dynamic World Dataset**:
   - **Source**: [Dynamic World - Google and the World Resources Institute (WRI)](https://dynamicworld.app/) 
   - **Description**:The Dynamic World dataset provides a near real-time, high-resolution (10-meter) global land cover classification. It is derived from Sentinel-2 imagery and utilizes machine learning models to classify land cover into nine distinct classes, including water, trees, grass, crops, built areas, bare ground, shrubs, flooded vegetation, and snow/ice. The dataset offers data with minimal latency, enabling near-immediate analysis and decision-making.
   - **Spatial Resolution**: 10 meters.
   - **Temporal Coverage**: Data is available since mid-2015, updated continuously as Sentinel-2 imagery becomes available capturing near Real-time.

2.**MODIS Dataset**:
   - **Source:** NASA's Moderate Resolution Imaging Spectroradiometer [MODIS](https://modis.gsfc.nasa.gov/data/) on Terra and Aqua satellites.
   - **Description:** The MODIS dataset provides a wide range of data products, including land surface temperature, vegetation indices, and land cover classifications. It is widely used for monitoring and modeling land surface processes.
   - **Spatial Resolution:** 250 meters.
   - **Temporal Coverage:** Data is available from 2000 to the present, with daily to 16-day composite products.

3. **Administrative Boundaries (HDX)**:
   - **Source**: Humanitarian Data Exchange [HDX](https://data.humdata.org/).
   - **Description**: Geographic boundaries used for spatial aggregation and administrative analysis, such as calculating productivity metrics by region (e.g., governorate or district).
   - **Use Case**: The administrative boundaries are used to aggregate EVI statistics by region and facilitate reporting at various administrative levels.


In [1]:
from pathlib import Path
import logging

import ee
import geemap
from gee_zonal import ZonalStats
import geopandas as gpd
import pandas as pd
import plotnine as p9
import altair as alt

logger = logging.getLogger(__name__)

PROJECT_ROOT = Path().cwd().parent.parent
DATA_PATH = PROJECT_ROOT / "data"
BOUNDARIES_PATH = DATA_PATH / "boundaries"

ee.Initialize()

In [2]:
def load_syria_boundaries(admin_level: int = 0) -> gpd.GeoDataFrame:
    """Load Syria administrative boundaries from shapefiles."""
    if admin_level not in [0, 1, 2, 3]:
        raise ValueError("admin_level must be 0, 1, 2, or 3")

    shapefile_path = BOUNDARIES_PATH / f"syr_admin{admin_level}.shp"
    return gpd.read_file(shapefile_path)


def bitwiseExtract(value: ee.Image, fromBit: int, toBit: int | None = None) -> ee.Image:
    """Extract bits from a binary image."""
    toBit = fromBit if toBit is None else toBit
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)


def apply_modisQA_mask(image: ee.Image):
    sqa = image.select("SummaryQA")
    dqa = image.select("DetailedQA")
    viQualityFlagsS = bitwiseExtract(sqa, 0, 1)
    viQualityFlagsD = bitwiseExtract(dqa, 0, 1)
    viSnowIceFlagsD = bitwiseExtract(dqa, 14)
    # Good data, use with confidence
    mask = (
        viQualityFlagsS.eq(0)
        .And(viQualityFlagsD.eq(0))
        .And(viQualityFlagsS.eq(1))
        .And(viQualityFlagsD.eq(1))
        .And(viSnowIceFlagsD)
        .eq(0)
    )
    return image.updateMask(mask)


def apply_scale_factor(image, scale_factor: float = 0.0001) -> ee.Image:
    """Apply scale factor to an image."""
    scaled_evi = image.multiply(scale_factor).copyProperties(
        image, ["system:time_start"]
    )
    return scaled_evi


def load_evi_data(
    start_date: str | ee.Date,
    end_date: str | ee.Date,
) -> ee.ImageCollection:
    """
    Load EVI data from MODIS.

    Args:
        start_date: Start date.
        end_date: End date.
        apply_crop_mask: Whether to apply crop mask.

    Returns:
        EVI image collection.
    """

    terra = (
        ee.ImageCollection("MODIS/061/MOD13Q1")
        .select(["EVI", "SummaryQA", "DetailedQA"])
        .filterDate(start_date, end_date)
    )

    aqua = (
        ee.ImageCollection("MODIS/061/MYD13Q1")
        .select(["EVI", "SummaryQA", "DetailedQA"])
        .filterDate(start_date, end_date)
    )

    mod13q1_QC = terra.map(apply_modisQA_mask)
    myd13q1_QC = aqua.map(apply_modisQA_mask)
    mxd13q1_cleaned = (
        mod13q1_QC.select("EVI").merge(myd13q1_QC.select("EVI")).map(apply_scale_factor)
    )
    mxd13q1 = mxd13q1_cleaned.sort("system:time_start")
    return mxd13q1

In [3]:
syria_adm0_shp = load_syria_boundaries(admin_level=0)
syria_adm1_shp = load_syria_boundaries(admin_level=1)
syria_adm2_shp = load_syria_boundaries(admin_level=2)
syria_adm3_shp = load_syria_boundaries(admin_level=3)

syria_adm0 = syria_adm0_shp.pipe(geemap.geopandas_to_ee)
syria_adm1 = syria_adm1_shp.pipe(geemap.geopandas_to_ee)
syria_adm2 = syria_adm2_shp.pipe(geemap.geopandas_to_ee)
syria_adm3 = syria_adm3_shp.pipe(geemap.geopandas_to_ee)

start_date = ee.Date(f"2010-01-01")
end_date = ee.Date(f"2025-08-30")

evi_data = load_evi_data(start_date, end_date)

In [4]:
def create_cropland_mask_by_year(year: int):
    """
    Create cropland mask for a specific year using Dynamic World land cover.
    For years before 2015, uses 2015 mask as Dynamic World only starts in 2015.
    """
    cropland_value = 4

    # Dynamic World starts in 2015
    mask_year = max(year, 2015)

    yearly_start = ee.Date.fromYMD(mask_year, 1, 1)
    yearly_end = ee.Date.fromYMD(mask_year, 12, 31)

    yearly_lc = (
        ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
        .filterDate(yearly_start, yearly_end)
        .filterBounds(syria_adm0)
        .select("label")
        # Get the most frequent land cover class
        .reduce(ee.Reducer.mode())
    )

    crop_mask = yearly_lc.eq(cropland_value)
    return crop_mask


def create_evi_masked_by_year(year: int):
    """Create EVI collection masked to cropland for a specific year."""
    crop_mask = create_cropland_mask_by_year(year)

    evi_subset = evi_data.filterDate(
        ee.Date.fromYMD(year, 1, 1), ee.Date.fromYMD(year, 12, 31)
    )

    def apply_crop_mask(image):
        return image.updateMask(crop_mask).set("year", year)

    evi_masked = evi_subset.map(apply_crop_mask)
    return evi_masked

## Admin level 0

In [5]:
run = False
if run:
    for year in range(2010, 2026):
        print(f"Processing year {year}...")

        try:
            evi_year = create_evi_masked_by_year(year)

            zs = ZonalStats(
                ee_dataset=evi_year,
                target_features=syria_adm0,
                scale=250,
                statistic_type="median",
                output_dir="Admin level 0",
                output_name=f"syria_adm0_evi_stats_{year}",
            )

            zs.runZonalStats()

            print(f"✓ Completed year {year}")

        except Exception as e:
            print(f"✗ Error processing year {year}: {str(e)}")
            continue

In [8]:
def preprocess_evi(evi_file: str | Path) -> pd.DataFrame:
    """Preprocess EVI CSV file."""
    evi_df = pd.read_csv(evi_file)

    metadata_cols = [
        col
        for col in evi_df.columns
        for word in ["PCODE", "_AR", "_EN", ".geo"]
        if word in col
    ]
    evi_df = (
        evi_df.rename(columns=lambda col: col[-14:] if col.endswith("_EVI") else col)
        .drop(columns=["system:index", "UPDATE_DAT"])
        .melt(
            id_vars=metadata_cols,
            var_name="band_date",
            value_name="EVI",
        )
        .assign(
            date=lambda df: pd.to_datetime(
                df["band_date"].str.extract(r"(\d{4}_\d{2}_\d{2})")[0],
                format="%Y_%m_%d",
            ),
        )
    )
    return evi_df


evi_adm0_df = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI 2010-2025"
                / "Admin level 0"
                / f"syria_adm0_evi_stats_{year}.csv"
            )
            for year in range(2010, 2026)
        ],
    )
    .drop(columns=[".geo"])
    .merge(syria_adm0_shp.filter(["PCODE", "geometry"]), on="PCODE", how="left")
    .set_index("date")
    .sort_index()
)

evi_adm0_df

,NAME_AR,NAME_EN,NAM_EN_REF,PCODE,band_date,EVI,geometry
date,,,,,,,
2010-01-01,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2010_01_01_EVI,0.165972,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."
2010-01-09,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2010_01_09_EVI,0.181585,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."
2010-01-17,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2010_01_17_EVI,0.212921,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."
2010-01-25,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2010_01_25_EVI,0.228479,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."
2010-02-02,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2010_02_02_EVI,0.232398,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."
...,...,...,...,...,...,...,...
2025-07-28,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2025_07_28_EVI,0.119073,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."
2025-08-05,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2025_08_05_EVI,0.122963,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."
2025-08-13,الجمهورية العربية السورية,Syrian Arab Republic,Syrian Arab Republic,SY,2025_08_13_EVI,0.122958,"MULTIPOLYGON (((35.85782 34.8591, 35.85969 34...."


In [ ]:
from scipy.signal import savgol_filter


def preprocess_series(series):
    """TIMESAT-style preprocessing."""
    series = series.interpolate(limit_direction="both")  # Step 2: Interpolate
    median = series.rolling(window=5, center=True).median()
    std_dev = series.std()
    series = series.mask(
        (series - median).abs() > 2 * std_dev
    )  # Step 1: Remove outliers
    series = series.interpolate(limit_direction="both")
    smoothed = savgol_filter(series, window_length=5, polyorder=2)  # Step 3: Smooth
    return smoothed


def extract_sos_mos_eos(smoothed, dates, threshold=0.2):
    max_val = smoothed.max()
    min_val = smoothed.min()
    amp = max_val - min_val
    sos = mos = eos = None
    for i in range(1, len(smoothed)):
        if sos is None and smoothed[i] > min_val + threshold * amp:
            sos = dates[i]
        if smoothed[i] == max_val:
            mos = dates[i]
        if sos is not None and smoothed[i] < min_val + threshold * amp:
            eos = dates[i]
            break
    return sos, mos, eos

In [ ]:
(
    evi_adm0_df.assign(month=lambda df: df.index.month)
    .groupby("month")
    .agg(EVI=("EVI", "mean"))
    .assign(smoothed=lambda df: preprocess_series(df["EVI"]))
    .assign(
        sos=lambda df: extract_sos_mos_eos(df["smoothed"].values, df.index)[0],
        mos=lambda df: extract_sos_mos_eos(df["smoothed"].values, df.index)[1],
        eos=lambda df: extract_sos_mos_eos(df["smoothed"].values, df.index)[2],
    )
    .reset_index()
    .pipe((p9.ggplot, "data"))
    + p9.aes(x="month")
    + p9.geom_line(p9.aes(y="EVI"))
    + p9.geom_point(p9.aes(y="smoothed"))
    + p9.geom_vline(p9.aes(xintercept="sos"), linetype="dashed", color="gray")
    + p9.geom_vline(p9.aes(xintercept="mos"), linetype="dashed", color="gray")
    + p9.geom_vline(p9.aes(xintercept="eos"), linetype="dashed", color="gray")
    + p9.theme_minimal()
    + p9.scale_x_continuous(breaks=range(1, 13))
    + p9.labs(title="Average EVI in Syria (2010-2025)", y="EVI", x="Month")
)

In [ ]:
monthly_evi = (
    evi_adm0_df.groupby(pd.Grouper(freq="MS"))
    .agg(EVI=("EVI", "median"))
    .assign(month=lambda x: x.index.month, year=lambda x: x.index.year)
)

(
    p9.ggplot(monthly_evi)
    + p9.aes(x="month", y="EVI", group="year")
    + p9.geom_line(alpha=0.5)
    + p9.theme_minimal()
    + p9.scale_x_continuous(breaks=range(1, 13))
    + p9.labs(title="Monthly Median EVI in Syria (2010-2025)")
)

In [ ]:
(
    evi_adm0_df.assign(year=lambda df: df.index.year)
    .groupby("year", as_index=False)
    .agg(EVI=("EVI", "median"))
    .pipe((p9.ggplot, "data"))
    + p9.aes(x="year", y="EVI")
    + p9.geom_line()
    + p9.theme_minimal()
    + p9.labs(title="Median EVI in Syria (2010-2025)", x="", y="EVI")
    + p9.theme(figure_size=(8, 4))
)

## Admin level 1

In [ ]:
run = False
if run:
    for year in range(2010, 2026):
        print(f"Processing year {year}...")

        try:
            evi_year = create_evi_masked_by_year(year)

            zs = ZonalStats(
                ee_dataset=evi_year,
                target_features=syria_adm1,
                scale=250,
                statistic_type="median",
                output_dir="Admin level 1",
                output_name=f"syria_adm1_evi_stats_{year}",
            )

            zs.runZonalStats()

            print(f"✓ Completed year {year}")

        except Exception as e:
            print(f"✗ Error processing year {year}: {str(e)}")
            continue

In [ ]:
evi_adm1_df = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI 2010-2025"
                / "Admin level 1"
                / f"syria_adm1_evi_stats_{year}.csv"
            )
            for year in range(2010, 2026)
        ],
    )
    .drop(columns=[".geo"])
    .merge(syria_adm1_shp.filter(["PCODE", "geometry"]), on="PCODE", how="left")
    .sort_values(["date", "NAME_EN"])
    .reset_index(drop=True)
    .assign(
        month=lambda df: df["date"].dt.to_period("M").dt.to_timestamp(),
        year=lambda df: df["date"].dt.to_period("Y").dt.to_timestamp(),
    )
)

evi_adm1_df

In [ ]:
(
    evi_adm1_df.groupby(["year", "PCODE", "NAME_EN"], as_index=False)
    .agg(EVI=("EVI", "median"))
    .assign(EVI=lambda df: df["EVI"].round(3))
    .pipe((alt.Chart, "data"))
    .mark_line(point=True)
    .encode(
        x="year:T",
        y="EVI:Q",
        tooltip=["year:T", "EVI:Q"],
        facet=alt.Facet("NAME_EN:N", columns=4),
    )
    .properties(width=200, height=150)
)

In [ ]:
(
    evi_adm1_df.assign(year=lambda df: df["year"].dt.year.astype(str))
    .groupby(["NAME_EN", "PCODE", "year"], as_index=False)
    .agg(EVI=("EVI", "median"))
    .assign(EVI=lambda df: df["EVI"].round(3))
    .rename(columns={"NAME_EN": "Name"})
    .pipe((alt.Chart, "data"))
    .mark_geoshape(stroke="white", strokeWidth=1.5)
    .encode(
        shape="geo:G",
        color="EVI:Q",
        tooltip=["Name:N", "EVI:Q"],
        facet=alt.Facet("year:N", columns=4),
    )
    .transform_lookup(
        lookup="PCODE",
        from_=alt.LookupData(data=syria_adm1_shp, key="PCODE"),
        as_="geo",
    )
    .properties(width=200, height=150)
)

## Admin level 2

In [ ]:
run = False
if run:
    for year in range(2010, 2026):
        print(f"Processing year {year}...")

        try:
            evi_year = create_evi_masked_by_year(year)

            zs = ZonalStats(
                ee_dataset=evi_year,
                target_features=syria_adm2,
                scale=250,
                statistic_type="median",
                output_dir="Admin level 2",
                output_name=f"syria_adm2_evi_stats_{year}",
            )

            zs.runZonalStats()

            print(f"✓ Completed year {year}")

        except Exception as e:
            print(f"✗ Error processing year {year}: {str(e)}")
            continue

In [ ]:
evi_adm2_df = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI 2010-2025"
                / "Admin level 2"
                / f"syria_adm2_evi_stats_{year}.csv"
            )
            for year in range(2010, 2026)
        ],
    )
    .drop(columns=[".geo"])
    .merge(syria_adm2_shp.filter(["PCODE", "geometry"]), on="PCODE", how="left")
    .sort_values(["date", "NAME_EN"])
    .assign(
        month=lambda df: df["date"].dt.to_period("M").dt.to_timestamp(),
        year=lambda df: df["date"].dt.to_period("Y").dt.to_timestamp(),
    )
)

In [ ]:
(
    evi_adm2_df.assign(year=lambda df: df["year"].dt.year.astype(str))
    .groupby(["NAME_EN", "PCODE", "year"], as_index=False)
    .agg(EVI=("EVI", "median"))
    .assign(EVI=lambda df: df["EVI"].round(3))
    .rename(columns={"NAME_EN": "Name"})
    .pipe((alt.Chart, "data"))
    .mark_geoshape(stroke="white", strokeWidth=1)
    .encode(
        shape="geo:G",
        color="EVI:Q",
        tooltip=["Name:N", "EVI:Q"],
        facet=alt.Facet("year:N", columns=4),
    )
    .transform_lookup(
        lookup="PCODE",
        from_=alt.LookupData(data=syria_adm2_shp, key="PCODE"),
        as_="geo",
    )
    .properties(width=200, height=150)
    .interactive()
)

## Admin level 3

In [ ]:
run = False
if run:
    for year in range(2010, 2026):
        print(f"Processing year {year}...")

        try:
            evi_year = create_evi_masked_by_year(year)

            zs = ZonalStats(
                ee_dataset=evi_year,
                target_features=syria_adm3,
                scale=250,
                statistic_type="median",
                output_dir="Admin level 3",
                output_name=f"syria_adm3_evi_stats_{year}",
            )

            zs.runZonalStats()

            print(f"✓ Completed year {year}")

        except Exception as e:
            print(f"✗ Error processing year {year}: {str(e)}")
            continue

In [ ]:
evi_adm3_df = (
    pd.concat(
        [
            preprocess_evi(
                DATA_PATH
                / "EVI 2010-2025"
                / "Admin level 3"
                / f"syria_adm3_evi_stats_{year}.csv"
            )
            for year in range(2010, 2026)
        ],
    )
    .drop(columns=[".geo"])
    .merge(syria_adm2_shp.filter(["PCODE", "geometry"]), on="PCODE", how="left")
    .sort_values(["date", "NAME_EN"])
    .reset_index(drop=True)
    .assign(
        month=lambda df: df["date"].dt.to_period("M").dt.to_timestamp(),
        year=lambda df: df["date"].dt.to_period("Y").dt.to_timestamp(),
    )
)

In [ ]:
(
    evi_adm3_df.assign(year=lambda df: df["year"].dt.year.astype(str))
    .groupby(["NAME_EN", "PCODE", "year"], as_index=False)
    .agg(EVI=("EVI", "median"))
    .assign(EVI=lambda df: df["EVI"].round(3))
    .rename(columns={"NAME_EN": "Name"})
    .pipe((alt.Chart, "data"))
    .mark_geoshape(stroke="white", strokeWidth=1)
    .encode(
        shape="geo:G",
        color="EVI:Q",
        tooltip=["Name:N", "EVI:Q"],
        facet=alt.Facet("year:N", columns=4),
    )
    .transform_lookup(
        lookup="PCODE",
        from_=alt.LookupData(data=syria_adm3_shp, key="PCODE"),
        as_="geo",
    )
    .properties(width=200, height=150)
    .interactive()
)